In [3]:
import pandas_dq
import pandas as pd
data = pd.read_csv('datasets\data.csv')
report = pandas_dq.dq_report(data, target=None, csv_engine="pandas", verbose=1)
reportJSON = report.to_json
#print(reportJSON)
reportDICT = report.to_dict()

Alert: Dropping 199 duplicate rows can sometimes cause column data types to change to object. Double-check!


C:\Users\robin\AppData\Local\Temp\ipykernel_19504\3798150608.py:4: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  report = pandas_dq.dq_report(data, target=None, csv_engine="pandas", verbose=1)


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Date,object,0.000000,16,2015-01-04,2017-08-06,"136 rare categories: Too many to list. Group them into a single category or drop the categories., high cardinality with 136 unique values: Use hash encoding or embedding to reduce dimension."
AveragePrice,float64,0.000000,NA,0.460000,2.750000,has 8 outliers greater than upper bound (2.4849999999999994) or lower than lower bound(0.2050000000000003). Cap them or remove them.
Total Volume,float64,0.000000,NA,385.550000,37130688.910000,has 98 outliers greater than upper bound (1262240.94) or lower than lower bound(-741600.02). Cap them or remove them.
4046,float64,0.000000,NA,0.000000,14699604.930000,"has 109 outliers greater than upper bound (292689.375) or lower than lower bound(-174153.145). Cap them or remove them., has a high correlation with ['Total Volume']. Consider dropping one of them."
4225,float64,0.000000,NA,0.000000,13926692.620000,"has 125 outliers greater than upper bound (417847.51999999996) or lower than lower bound(-245488.23999999996). Cap them or remove them., has a high correlation with ['Total Volume', '4046']. Consider dropping one of them."
4770,float64,0.000000,NA,0.000000,1326422.560000,"has 107 outliers greater than upper bound (23569.274999999998) or lower than lower bound(-14141.564999999999). Cap them or remove them., has a high correlation with ['Total Volume', '4046', '4225']. Consider dropping one of them."
Total Bags,float64,0.000000,NA,0.000000,10863029.040000,"has 119 outliers greater than upper bound (293542.67) or lower than lower bound(-169645.08999999997). Cap them or remove them., has a high correlation with ['Total Volume', '4046', '4225']. Consider dropping one of them."
Small Bags,float64,0.000000,NA,0.000000,8555283.680000,"has 115 outliers greater than upper bound (216211.72) or lower than lower bound(-126383.96). Cap them or remove them., has a high correlation with ['Total Volume', '4046', '4225', '4770', 'Total Bags']. Consider dropping one of them."
Large Bags,float64,0.000000,NA,0.000000,2214419.290000,"has 133 outliers greater than upper bound (55577.945) or lower than lower bound(-33237.615). Cap them or remove them., has a high correlation with ['Total Bags']. Consider dropping one of them."
XLarge Bags,float64,0.000000,NA,0.000000,93326.070000,has 182 outliers greater than upper bound (191.3) or lower than lower bound(-114.78). Cap them or remove them.


In [5]:
from pandas_dq import Fix_DQ
fdq = Fix_DQ()

X_train_transformed = fdq.fit_transform(data)


Alert: Detecting 199 duplicate rows...
    Dropping 4046 which has a high correlation with ['Total Volume']
    Dropping 4225 which has a high correlation with ['Total Volume', '4046']
    Dropping 4770 which has a high correlation with ['Total Volume', '4046', '4225']
    Dropping Total Bags which has a high correlation with ['Total Volume', '4046', '4225']
    Dropping Small Bags which has a high correlation with ['Total Volume', '4046', '4225', '4770', 'Total Bags']
    Dropping Large Bags which has a high correlation with ['Total Bags']
Alert: Dropping 199 duplicate rows can sometimes cause column data types to change to object. Double-check!


C:\Users\robin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:878: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  return self.fit(X, **fit_params).transform(X)


In [6]:
report = pandas_dq.dq_report(X_train_transformed, target=None, csv_engine="pandas", verbose=1)

Alert: Dropping 16 duplicate rows can sometimes cause column data types to change to object. Double-check!


C:\Users\robin\AppData\Local\Temp\ipykernel_19504\3440526732.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  report = pandas_dq.dq_report(X_train_transformed, target=None, csv_engine="pandas", verbose=1)


,Data Type,Missing Values%,Unique Values%,Minimum Value,Maximum Value,DQ Issue
Date,object,0.000000,0,Rare,Rare,Zero-variance colum: drop before modeling process.
AveragePrice,float64,0.000000,NA,0.460000,2.750000,has 7 outliers greater than upper bound (2.51) or lower than lower bound(0.19000000000000028). Cap them or remove them.
Total Volume,float64,0.000000,NA,5.957261,13.930087,No issue
XLarge Bags,float64,0.000000,NA,-0.637999,1.425258,No issue
type,object,0.000000,0,conventional,organic,No issue
year,int64,0.000000,0,2015,2017,No issue
region,object,0.000000,0,Rare,Rare,Zero-variance colum: drop before modeling process.


In [8]:
import cleanlab
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv('datasets\Iris.csv')
# Create a label encoder object
le = LabelEncoder()
# Fit and transform the "Species" column
df['Species'] = le.fit_transform(df['Species'])
def cleanlab_test(df):

    # cleanlab works with **any classifier**. Yup, you can use PyTorch/TensorFlow/OpenAI/XGBoost/etc.
    cl = cleanlab.classification.CleanLearning(RandomForestClassifier())
    data = df.loc[:, df.columns != 'class']

    # cleanlab finds data and label issues in **any dataset**... in ONE line of code!
    # Fit model to messy, real-world data, automatically training on cleaned data.
    _ = cl.fit(data, df['class'])
    # See the label quality for every example, which data has issues, and more.
    print(cl.get_label_issues())

In [66]:
from deepchecks.tabular import Dataset
import deepchecks.tabular.checks
import pandas as pd
from sklearn.datasets import load_iris
from deepchecks.tabular.datasets.classification.phishing import load_data
import numpy as np
from deepchecks.tabular.datasets.classification import adult
import plotly.express as px
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [93]:
def encode_categorical_columns(dataset, target, data_types):
    # Find all categorical columns
    # TODO: regel hieronder aanpassen naar wat de user heeft ingegeven (let hierbij op strings als dtype, moeten gezien worden als categorical)
    categorical_cols = dataset.select_dtypes(include=['object', 'category']).columns.tolist()

    target_is_categorical = False
    if target != 'None':
        # remove target as we want to label encode this (for classification problems)
        if target in categorical_cols:
            target_is_categorical = True
            categorical_cols.remove(target)

            # label encode target
            le = LabelEncoder()
            encoded_target = le.fit_transform(dataset[target])
            # replace target column with label encoded values
            dataset.drop(columns=[target], inplace=True)
            dataset[target] = encoded_target

    if not categorical_cols:  # then no features are categorical, and we're done
        return dataset

    # if there are categorical columns, we want to one-hot-encode them

    # encode categoricals
    encoder = OneHotEncoder(handle_unknown='ignore', max_categories=100)
    encoded_columns = encoder.fit_transform(dataset[categorical_cols])
    new_columns = pd.DataFrame(encoded_columns.toarray(), columns=encoder.get_feature_names_out(categorical_cols))

    # add new columns to df and drop old ones
    dataset_encoded = pd.concat([dataset, new_columns], axis=1)
    dataset_encoded = dataset_encoded.drop(columns=categorical_cols)

    # reposition target column to the end of the dataframe
    if target != 'None' and target_is_categorical:
        dataset_encoded.drop(columns=[target], inplace=True)
        dataset_encoded[target] = encoded_target

    #XGBClassifier doesn't accept: [, ] or <, so loop over the columns and change the names if they contain such values
    new_col_names = {col: col.replace('<', '(smaller than)').replace('[', '(').replace(']', ')') for col in dataset_encoded.columns}
    dataset_encoded = dataset_encoded.rename(columns=new_col_names)

    return dataset_encoded

In [125]:
def wrong_label(encoded_dataset, target):
    """"Function that finds potential label errors (due to annotator mistakes), edge cases, and otherwise ambiguous examples"""
    model_XGBC = XGBClassifier(tree_method="hist", enable_categorical=True) #hist is fastest tree method of XGBoost
    #TODO: add more models and choose whichever gives highest prediction? Downside --> slows process
    data_no_labels = encoded_dataset.drop(columns=[target])
    labels = encoded_dataset[target]
    pred_probs = cross_val_predict(model_XGBC, data_no_labels, labels, method='predict_proba')

    preds = np.argmax(pred_probs, axis=1)
    acc_original = accuracy_score(preds, labels)
    print(f"Accuracy with original data: {round(acc_original * 100, 1)}%")

    #use cleanlab's built in confident learning method to find label issues
    cl = cleanlab.classification.CleanLearning()
    issues_dataframe = cl.find_label_issues(X=None, labels=labels, pred_probs=pred_probs)
    wrong_label_count = (issues_dataframe['is_label_issue'] == True).sum()

    #filter df so only errors are visible
    issues_dataframe_only_errors = issues_dataframe[issues_dataframe['is_label_issue'] == True]

    return issues_dataframe, issues_dataframe_only_errors, wrong_label_count

In [126]:
from scipy.io.arff import loadarff

raw_data = loadarff('datasets/dataset_31_credit-g.arff')
df_data = pd.DataFrame(raw_data[0])
#df_data.drop(columns=['checking_status', 'savings_status', 'employment'], inplace=True)
encdoded_data = encode_categorical_columns(df_data, 'class', None)
# pred_probs, labels = wrong_label(encdoded_data, 'class')

wrong_label(encdoded_data, 'class')

# cl = cleanlab.classification.CleanLearning()
# issues_dataframe = cl.find_label_issues(X=None, labels=labels, pred_probs=pred_probs, return_indices_ranked_by='self_confidence')
# print(issues_dataframe)

Accuracy with original data: 75.7%
   is_label_issue  label_quality  given_label  predicted_label
0           False       0.982974            1                1
1           False       0.896697            0                0
2           False       0.995315            1                1
3            True       0.405909            1                0
4           False       0.830484            0                0


(     is_label_issue  label_quality  given_label  predicted_label
 0             False       0.982974            1                1
 1             False       0.896697            0                0
 2             False       0.995315            1                1
 3              True       0.405909            1                0
 4             False       0.830484            0                0
 ..              ...            ...          ...              ...
 995           False       0.995562            1                1
 996           False       0.788997            1                1
 997           False       0.996626            1                1
 998           False       0.964832            0                0
 999           False       0.864109            1                1
 
 [1000 rows x 4 columns],
 200)